# Tarefa 02 Módulo 05

O nosso projeto desta sequência de módulos do curso será um aprofundamento da demonstração sobre classificação de risco de crédito que vimos lá no comecinho. Pois recebemos uma base já montada pra nós. Tenha certeza de que ela passou por um longo processamento até ficar daquele jeito. Neste exercício vamos exercitar o que aprendemos nas ultimas aulas e montar a variável resposta da base do nosso projeto.

#### Marcação de bom e mau
O objetivo da modelagem é classificar o risco de inadimplência, ou como se diz no meio, o risco de *default*. Podemos fazer longas discussões sobre o conceito de *default* com base em estudos e exigências regulatórias, para efeitos deste estudo, um cliente em *default* é aquele que está em 60 dias de atraso ou mais. Então classificaremos os clientes como 'bons' e 'maus' assim:
- **Maus** pagadores: são aqueles que entraram em 'default' (atraso 60 dias ou mais) nos 24 meses seguintes à aquisição do cartão de crédito. 
- **Bons** pagadores: são considerados todos os demais.
- **Excluídos**: Clientes que não adquiriram um cartão de crédito (seja por recusa, seja por desistência) não possuem informações de pagamento, portanto não se pode identificar se são bons ou maus. Há uma longa discussão e literatura sobre *inferência de rejeitados* que está fora do escopo deste exercício.

#### Bases disponíveis
Temos duas bases importantes aqui: uma de propostas, com diversas informações dos vários solicitantes de cartão de crédito, e uma base de pagamentos. A base de pagamentos será utilizada para identificar a ocorrência de *default*. A base de propostas tem diversas informações coletadas no momento da solicitação do crédito (isto é importante: qualquer informação posterior a essa data é impossível de ser coletada na aplicação do modelo e não pode ser utilizada).

As variáveis delas são:

Base de propostas - application_records.csv

| Nome da Variável         | Description                                         | Tipo  |
| ------------------------ |:---------------------------------------------------:| -----:|
| ID| identificador do cliente (chave) |inteiro|
| CODE_GENDER| M = 'Masculino'; F = 'Feminino' |M/F|
| FLAG_OWN_CAR| Y = 'possui'; N = 'não possui' |Y/N|
| FLAG_OWN_REALTY| Y = 'possui'; N = 'não possui' |Y/N|
| CNT_CHILDREN| Quantidade de filhos |inteiro|
| AMT_INCOME_TOTAL| Annual income |inteiro|
| NAME_INCOME_TYPE|Tipo de renda (ex: assaliariado, autônomo etc) | texto |
| NAME_EDUCATION_TYPE| Nível de educação (ex: secundário, superior etc) |texto|
| NAME_FAMILY_STATUS | Estado civil (ex: solteiro, casado etc)| texto |
| NAME_HOUSING_TYPE | tipo de residência (ex: casa/apartamento, com os pais etc) | texto |
| DAYS_BIRTH | Count backwards from current day (0), -1 means yesterday |inteiro|
| DAYS_EMPLOYED | Count backwards from current day (0), -1 means yesterday |inteiro|
| FLAG_MOBIL | Indica se possui celular (1 = sim, 0 = não) |binária|
| FLAG_WORK_PHONE | Indica se possui telefone comercial (1 = sim, 0 = não) |binária|
| FLAG_PHONE | Indica se possui telefone (1 = sim, 0 = não) |binária|
| FLAG_EMAIL | Indica se possui e-mail (1 = sim, 0 = não) |binária|
| OCCUPATION_TYPE | Occupation	 |Qualitativa|
| CNT_FAM_MEMBERS | quantidade de pessoas na residência |inteiro|

Base de pagamentos - pagamentos_largo.csv  

| Nome da Variável         | Description                                         | Tipo  |
| ------------------------ |:---------------------------------------------------:| -----:|
| ID| identificador do cliente (chave) |inteiro|
| mes_00 a mes_24| faixa de atraso mês a mês do cliente <br>0: 1-29 days past due &nbsp;&nbsp;&nbsp;&nbsp; 1: 30-59 days past due <br />2: 60-89 days overdue &nbsp;&nbsp;&nbsp;&nbsp; 3: 90-119 days overdue <br /> 4: 120-149 days overdue &nbsp;&nbsp;&nbsp;&nbsp; 5: more than 150 days <br />C: paid off that month &nbsp;&nbsp;&nbsp;&nbsp; X: No loan for the month |Qualitativa|

#### Construindo a variável resposta
A base de pagamentos está em um formato de 'base larga'. Essa base possui informações de pagamentos do cliente mês a mês a partir do mês de aquisição do crédito (mês 0) até o vigésimo quarto mês após a aquisição do crédito (mês 24). Utilizaremos essa base para determinar se um proponente é considerado 'bom pagador' ou caso apresente atraso representativo, será considerado 'mau pagador'.

#### Base larga vs base longa
A base ser larga significa que há uma linha para cada cliente, e que as informações estarão nas colunas, em contraste com a 'base longa', em que haveria uma linha para cada combinação cliente/mês, uma coluna indicando o cliente, outra indicando o mês, e apenas uma coluna com a informação do atraso.

#### Tarefa 1) Marcar *default* no mês
Faça uma indicadora de se o cliente está em *default* em cada uma das marcações (mes_00 a mes_24). Dica: você pode utilizar o método ```.isin()``` do Pandas. Consulte a [documentação](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.isin.html) caso necessário.

#### Tarefa 2) 'bons' e 'maus' ao longo de todos os 24 meses de desempenho
Marque para cada cliente se ele teve pelo menos um episódio de *default* entre o mês 0 e o mês 24. Dica: o método ```sum()``` pode ajudar. Caso precise, consulte a [documentação](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sum.html) e procure pelo argumento ```axis```, você viu outros métodos que possuem esse argumento também. Tendo o número de meses em default de cada cliente, basta marcar ```True``` para todos aqueles que possuem pelo menos 1 mês em *default* e ```False``` para os demais.

#### Tarefa 3) Marcando proponentes expostos ao risco de crédito
Marcando proponentes que se tornaram tomadores: lembre-se de que clientes que não adquiriram o cartão devem ser desconsiderados. A base de pagamentos possui apenas clientes que adquiriram cartão de crédito, então você pode selecionar somente os clientes da base de propostas que se encontram na base de pagamentos.

#### Tarefa 4) Consolidando as informações
Faça uma junção das informações da base de propostas com a variável de *default* que você acabou de construir. Talvez você consiga realizar a tarefa 3 e tarefa 4 em uma única linha de código ;)

#### Tarefa 5) Verificando
Faça uma contagem dos valores do *default* que você construiu. 

-------------------------------------------------------------------

In [50]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)

In [51]:
propostas = pd.read_csv('application_record.csv')
pg = pd.read_csv('pagamentos_largo.csv')

In [52]:
pg.fillna('0', inplace = True)

In [53]:
pg.head()

,ID,mes_0,mes_1,mes_10,mes_11,mes_12,mes_13,mes_14,mes_15,mes_16,mes_17,mes_18,mes_19,mes_2,mes_20,mes_21,mes_22,mes_23,mes_24,mes_3,mes_4,mes_5,mes_6,mes_7,mes_8,mes_9
0,5001718,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,5001719,0,0,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C
2,5001720,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0
3,5001723,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,5001726,0,0,C,C,C,C,C,C,C,C,C,C,0,C,C,C,C,C,0,0,0,C,C,C,C


In [54]:
# 1) Seu código aqui
pg_default = pg.isin(['2', '3', '4', '5'])

In [55]:
pg_default

,ID,mes_0,mes_1,mes_10,mes_11,mes_12,mes_13,mes_14,mes_15,mes_16,mes_17,mes_18,mes_19,mes_2,mes_20,mes_21,mes_22,mes_23,mes_24,mes_3,mes_4,mes_5,mes_6,mes_7,mes_8,mes_9
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20932,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
20933,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
20934,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
20935,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [56]:
# 2) Seu código aqui
pg['inadimplente'] = 'not default'

In [57]:
pg.shape

(20937, 27)

In [58]:
pg.head()

,ID,mes_0,mes_1,mes_10,mes_11,mes_12,mes_13,mes_14,mes_15,mes_16,mes_17,mes_18,mes_19,mes_2,mes_20,mes_21,mes_22,mes_23,mes_24,mes_3,mes_4,mes_5,mes_6,mes_7,mes_8,mes_9,inadimplente
0,5001718,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,not default
1,5001719,0,0,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,not default
2,5001720,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,not default
3,5001723,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,not default
4,5001726,0,0,C,C,C,C,C,C,C,C,C,C,0,C,C,C,C,C,0,0,0,C,C,C,C,not default


In [59]:
pg['inadimplente'].value_counts()

not default    20937
Name: inadimplente, dtype: int64

In [60]:
# Atualizando a coluna inadimplente

tamanho = len(pg.columns)

for i in range(tamanho - 2):

    pg.loc[pg[f'mes_{i}'] == '2', 'inadimplente'] = 'default'
    pg.loc[pg[f'mes_{i}'] == '3', 'inadimplente'] = 'default'
    pg.loc[pg[f'mes_{i}'] == '4', 'inadimplente'] = 'default'
    pg.loc[pg[f'mes_{i}'] == '5', 'inadimplente'] = 'default'

    print(f'filtrando coluna mes_{i}')
    

filtrando coluna mes_0
filtrando coluna mes_1
filtrando coluna mes_2
filtrando coluna mes_3
filtrando coluna mes_4
filtrando coluna mes_5
filtrando coluna mes_6
filtrando coluna mes_7
filtrando coluna mes_8
filtrando coluna mes_9
filtrando coluna mes_10
filtrando coluna mes_11
filtrando coluna mes_12
filtrando coluna mes_13
filtrando coluna mes_14
filtrando coluna mes_15
filtrando coluna mes_16
filtrando coluna mes_17
filtrando coluna mes_18
filtrando coluna mes_19
filtrando coluna mes_20
filtrando coluna mes_21
filtrando coluna mes_22
filtrando coluna mes_23
filtrando coluna mes_24


In [61]:
pg['inadimplente'].value_counts()

not default    20506
default          431
Name: inadimplente, dtype: int64

In [62]:
# 3 e 4) Seu código aqui
pg_propostas =  pg.merge(propostas, on = 'ID', how = 'inner')

In [63]:
pg_propostas.sample(25)

,ID,mes_0,mes_1,mes_10,mes_11,mes_12,mes_13,mes_14,mes_15,mes_16,mes_17,mes_18,mes_19,mes_2,mes_20,mes_21,mes_22,mes_23,mes_24,mes_3,mes_4,mes_5,mes_6,mes_7,mes_8,mes_9,inadimplente,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS
8119,5068745,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,not default,F,N,Y,3,225000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-11870,-5072,1,0,0,0,Laborers,4.0
13709,5125291,0,0,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,not default,F,N,N,1,202500.0,Working,Secondary / secondary special,Separated,House / apartment,-13939,-134,1,0,0,0,Laborers,2.0
8256,5069119,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,not default,F,Y,Y,0,103500.0,Pensioner,Higher education,Civil marriage,House / apartment,-23002,365243,1,0,1,0,NaN,2.0
5320,5050534,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,not default,M,N,N,2,225000.0,Working,Higher education,Civil marriage,House / apartment,-16768,-3088,1,1,0,0,Laborers,4.0
14564,5135663,0,0,C,C,C,C,C,C,C,C,C,C,0,C,C,C,C,C,0,0,0,0,0,0,0,not default,F,Y,Y,0,99000.0,Pensioner,Higher education,Single / not married,House / apartment,-18352,365243,1,0,1,0,NaN,1.0
12041,5113327,0,0,C,C,C,C,C,C,C,C,C,C,0,C,C,C,C,C,0,0,C,C,C,C,C,not default,F,N,Y,0,180000.0,Pensioner,Secondary / secondary special,Married,House / apartment,-22381,365243,1,0,0,0,NaN,2.0
1906,5024170,0,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,not default,F,N,Y,0,202500.0,Pensioner,Secondary / secondary special,Civil marriage,House / apartment,-22042,365243,1,0,0,0,NaN,2.0
1945,5024280,0,1,C,C,C,C,C,C,C,C,C,C,0,C,C,C,C,C,0,0,0,C,C,C,C,not default,M,Y,Y,0,315000.0,Commercial associate,Secondary / secondary special,Married,House / apartment,-15509,-1189,1,1,1,0,Drivers,2.0
317,5009731,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,not default,M,Y,N,0,292500.0,Working,Secondary / secondary special,Married,House / apartment,-13178,-1317,1,1,0,0,Laborers,2.0
10153,5091875,0,0,C,C,C,C,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,C,C,not default,F,Y,Y,0,202500.0,Commercial associate,Secondary / secondary special,Married,House / apartment,-18463,-7048,1,0,0,0,Accountants,2.0


In [64]:
# 5) eu código aqui
pg_propostas['inadimplente'].value_counts()

not default    16260
default          390
Name: inadimplente, dtype: int64